In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# As a part of starter notebook would be running some data visualizations to understand the data and put in some insights

In [ ]:
train_data = pd.read_csv("../input/tabular-playground-series-apr-2021/train.csv")
test_data = pd.read_csv("../input/tabular-playground-series-apr-2021/test.csv")

In [ ]:
train_data.head()

In [ ]:
train_data.describe()

# Visualizing using AutoViz

In [ ]:
!pip install autoviz
!pip install xlrd

In [ ]:
#importing Autoviz class
from autoviz.AutoViz_Class import AutoViz_Class
#Instantiate the AutoViz class
AV = AutoViz_Class()
filepath = '../input/tabular-playground-series-apr-2021/train.csv'
df = AV.AutoViz(filepath, depVar='Survived', header=1)

# Lable Encoding and imputing the data

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder

cat_columns = ['Sex','Embarked']

label_train = train_data.copy()
label_test = test_data.copy()

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in cat_columns:
    label_train[col] = label_encoder.fit_transform(train_data[col])
    label_test[col] = label_encoder.transform(test_data[col])

In [ ]:
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']

In [ ]:
from sklearn.impute import SimpleImputer

# Imputation
simple_impute = SimpleImputer()
imputed_X_train = pd.DataFrame(simple_impute.fit_transform(label_train[features]))
imputed_X_test = pd.DataFrame(simple_impute.transform(label_test[features]))

# Imputation removed column names; put them back
imputed_X_train.columns = label_train[features].columns
imputed_X_test.columns = label_test[features].columns

y_train_data = label_train['Survived']

In [ ]:
from sklearn.feature_selection import mutual_info_classif as mif

In [ ]:
mif(imputed_X_train,y_train_data)

# Running some base Models

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
models = [DecisionTreeClassifier(),RandomForestClassifier(),KNeighborsClassifier(),GradientBoostingClassifier(),LGBMClassifier(),XGBClassifier()]

for model in models:
    X_train, X_valid, y_train, y_valid = train_test_split(imputed_X_train,y_train_data,test_size=0.1,random_state=42)
    model.fit(X_train, y_train)
    accuracy = accuracy_score(y_valid,model.predict(X_valid))
    print("Model : {0} Accuracy : {1}".format(model,accuracy))

**Base model zeroed down to LGBM as it gives best accuracy on validation**

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(imputed_X_train,y_train_data,test_size=0.1,random_state=42)
model = LGBMClassifier()
model.fit(X_train, y_train)
accuracy = accuracy_score(y_valid,model.predict(X_valid))
print("Model : {0} Accuracy : {1}".format(model,accuracy))
y_pred = model.predict(imputed_X_test)

In [ ]:
submission = pd.read_csv("../input/tabular-playground-series-apr-2021/sample_submission.csv")

In [ ]:
submission['Survived'] = y_pred

In [ ]:
submission.to_csv("submission.csv",index=False)

Will be tuning the hyperparameters for the LGBM model and also do some feature improvements if possible in the next notebook